In [ ]:
import kagglehub
deepcontractor_car_price_prediction_challenge_path = kagglehub.dataset_download('deepcontractor/car-price-prediction-challenge')

print('Data source import complete.')


Data source import complete.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/car-price-prediction-challenge/car_price_prediction.csv


# Data Importing

In [ ]:
df=pd.read_csv("/kaggle/input/car-price-prediction-challenge/car_price_prediction.csv")



```
# This is formatted as code
```

# Data cleaning

In [ ]:
df.head(20)

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
5,45802912,39493,891,HYUNDAI,Santa FE,2016,Jeep,Yes,Diesel,2,160931 km,4.0,Automatic,Front,04-May,Left wheel,White,4
6,45656768,1803,761,TOYOTA,Prius,2010,Hatchback,Yes,Hybrid,1.8,258909 km,4.0,Automatic,Front,04-May,Left wheel,White,12
7,45816158,549,751,HYUNDAI,Sonata,2013,Sedan,Yes,Petrol,2.4,216118 km,4.0,Automatic,Front,04-May,Left wheel,Grey,12
8,45641395,1098,394,TOYOTA,Camry,2014,Sedan,Yes,Hybrid,2.5,398069 km,4.0,Automatic,Front,04-May,Left wheel,Black,12
9,45756839,26657,-,LEXUS,RX 350,2007,Jeep,Yes,Petrol,3.5,128500 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

In [ ]:
df.shape

(19237, 18)

In [ ]:
columns = df.columns
for col in columns:
    print(f"Unique values in column '{col}':")
    print(df[col].unique())
    print("-" * 30)

Unique values in column 'ID':
[45654403 44731507 45774419 ... 45804997 45793526 45813273]
------------------------------
Unique values in column 'Price':
[13328 16621  8467 ... 56814 63886 22075]
------------------------------
Unique values in column 'Levy':
['1399' '1018' '-' '862' '446' '891' '761' '751' '394' '1053' '1055'
 '1079' '810' '2386' '1850' '531' '586' '1249' '2455' '583' '1537' '1288'
 '915' '1750' '707' '1077' '1486' '1091' '650' '382' '1436' '1194' '503'
 '1017' '1104' '639' '629' '919' '781' '530' '640' '765' '777' '779' '934'
 '769' '645' '1185' '1324' '830' '1187' '1111' '760' '642' '1604' '1095'
 '966' '473' '1138' '1811' '988' '917' '1156' '687' '11714' '836' '1347'
 '2866' '1646' '259' '609' '697' '585' '475' '690' '308' '1823' '1361'
 '1273' '924' '584' '2078' '831' '1172' '893' '1872' '1885' '1266' '447'
 '2148' '1730' '730' '289' '502' '333' '1325' '247' '879' '1342' '1327'
 '1598' '1514' '1058' '738' '1935' '481' '1522' '1282' '456' '880' '900'
 '798' '1277' '

In [ ]:
for col in df.select_dtypes(include='object').columns:
    print(f"\nValue counts for '{col}':")
    print(df[col].value_counts())


Value counts for 'Levy':
Levy
-       5819
765      486
891      461
639      410
640      405
        ... 
2308       1
4860       1
1641       1
1045       1
5681       1
Name: count, Length: 559, dtype: int64

Value counts for 'Manufacturer':
Manufacturer
HYUNDAI          3769
TOYOTA           3662
MERCEDES-BENZ    2076
FORD             1111
CHEVROLET        1069
                 ... 
LAMBORGHINI         1
PONTIAC             1
SATURN              1
ASTON MARTIN        1
GREATWALL           1
Name: count, Length: 65, dtype: int64

Value counts for 'Model':
Model
Prius              1083
Sonata             1079
Camry               938
Elantra             922
E 350               542
                   ... 
530 i                 1
E 500 AVG             1
Vito Extralong        1
C 240 W 203           1
Vito Exstralong       1
Name: count, Length: 1590, dtype: int64

Value counts for 'Category':
Category
Sedan          8736
Jeep           5473
Hatchback      2847
Minivan         647
Coup

## Find null values

In [ ]:
df.isna().sum()

,0
ID,0
Price,0
Levy,0
Manufacturer,0
Model,0
Prod. year,0
Category,0
Leather interior,0
Fuel type,0
Engine volume,0


## Find duplicated rows

In [ ]:
df.duplicated().value_counts()

,count
False,18924
True,313


In [ ]:
df = df.drop_duplicates()


In [ ]:
df.duplicated().value_counts()

,count
False,18924


## Categorical and numerical data

In [ ]:
numeric_col = [col for col in df.columns if df[col].dtype != 'object']
categorical_col = [col for col in df.columns if df[col].dtype == 'object']

In [ ]:
print(numeric_col)

['ID', 'Price', 'Prod. year', 'Cylinders', 'Airbags']


In [ ]:
print(categorical_col)

['Levy', 'Manufacturer', 'Model', 'Category', 'Leather interior', 'Fuel type', 'Engine volume', 'Mileage', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color']


# EDA

In [ ]:
fig = px.histogram(df, x='Prod. year', nbins=30,
                   title='Distribution of Production Years',
                   labels={'Prod. year': 'Production Year'},
                   template="plotly_white")
fig.show()

The production years show a clear upward trend, with the most significant volume of production occurring in recent years, particularly between 2010 and 2014, accounting for 9179 items. Production before the 1990s was negligible.

In [ ]:
manufacturer_counts = df['Manufacturer'].value_counts().reset_index()
manufacturer_counts.columns = ['Manufacturer', 'Count']
top_manufacturers = manufacturer_counts.head(10)

fig = px.bar(top_manufacturers, y='Manufacturer', x='Count', orientation='h',
             title='Top 10 Car Manufacturers',
             labels={'Manufacturer': 'Manufacturer', 'Count': 'Number of Cars'},
             template="plotly_white")
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

Hyundai and Toyota are the dominant car manufacturers, significantly outperforming others in terms of the number of cars. European luxury brands like Mercedes-Benz and BMW also feature prominently, while Nissan and Volkswagen appear to have the lowest representation among the top 10.

In [ ]:
category_counts = df['Category'].value_counts().reset_index()
category_counts.columns = ['Category', 'Count']

fig = px.bar(category_counts, x='Category', y='Count',
             title='Number of Cars per Category',
             labels={'Category': 'Category', 'Count': 'Number of Cars'},
             template="plotly_white")
fig.show()

Sedans overwhelmingly dominate the car categories, followed by Jeeps, while Hatchbacks also represent a significant portion. Minivans, Coupes, and Universal cars have a moderate presence, and specialized categories like Pickup, Cabriolet, and Limousine are the least common.

In [ ]:
top_manufacturers_list = df['Manufacturer'].value_counts().index[:15].tolist()
df_filtered_manufacturers = df[df['Manufacturer'].isin(top_manufacturers_list)].copy()

fig = px.box(df_filtered_manufacturers, y='Manufacturer', x='Price',
             title='Price Distribution by Manufacturer',
             labels={'Manufacturer': 'Manufacturer', 'Price': 'Price (GEL)'},
             color='Manufacturer',
             template="plotly_white",
             category_orders={"Manufacturer": top_manufacturers_list})
fig.update_layout(height=500)
fig.show()

The "Price Distribution by Manufacturer" chart provides limited actionable insights due to most manufacturers having their prices tightly clustered near the low end of the scale. The presence of an extreme outlier for Opel (around 26 million GEL) heavily skews the visual, making it difficult to analyze typical price ranges for other brands.

In [ ]:
fig = px.box(df, x='Fuel type', y='Price',
             title='Price Distribution by Fuel Type',
             labels={'Fuel type': 'Fuel Type', 'Price': 'Price (GEL)'},
             color='Fuel type',
             template="plotly_white")
fig.show()

The "Price Distribution by Fuel Type" chart indicates that most fuel types, including Hybrid, Petrol, Diesel, CNG, Plug-in Hybrid, LPG, and Hydrogen, generally have prices clustered at the lower end of the spectrum. However, there is a very high outlier price associated with Diesel, suggesting an exceptionally expensive diesel vehicle or data anomaly, which significantly extends its visible price range compared to other fuel types.

## Detect price outlier

In [ ]:
fig_box_filtered = px.box(df, y="Price", title="Box Plot of Price")
fig_box_filtered.show()

# Data preprocessing

In [ ]:
df.drop(['ID','Doors'], axis=1, inplace=True)

drop ID and doors columns

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18924 entries, 0 to 19236
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             18924 non-null  int64  
 1   Levy              18924 non-null  object 
 2   Manufacturer      18924 non-null  object 
 3   Model             18924 non-null  object 
 4   Prod. year        18924 non-null  int64  
 5   Category          18924 non-null  object 
 6   Leather interior  18924 non-null  object 
 7   Fuel type         18924 non-null  object 
 8   Engine volume     18924 non-null  object 
 9   Mileage           18924 non-null  object 
 10  Cylinders         18924 non-null  float64
 11  Gear box type     18924 non-null  object 
 12  Drive wheels      18924 non-null  object 
 13  Wheel             18924 non-null  object 
 14  Color             18924 non-null  object 
 15  Airbags           18924 non-null  int64  
dtypes: float64(1), int64(3), object(12)
memory us

## Change categorical data to numerical data

In [ ]:
df['Levy'] = df['Levy'].replace('-', np.nan)
df['Levy'] = pd.to_numeric(df['Levy'], errors='coerce')
df['Levy'].fillna(df['Levy'].median(), inplace=True)

/tmp/ipython-input-690-3626290181.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
df['Levy'].value_counts()

,count
Levy,
781.0,6003
765.0,482
891.0,453
639.0,403
640.0,398
...,...
2308.0,1
4860.0,1
1641.0,1


In [ ]:
df['Engine volume'] = df['Engine volume'].astype(str).str.replace(r'[^\d.]', '', regex=True)
df['Engine volume'] = pd.to_numeric(df['Engine volume'], errors='coerce')
df['Engine volume'].fillna(df['Engine volume'].median(), inplace=True)

In [ ]:
df['Mileage'] = df['Mileage'].astype(str).str.replace(' km', '', regex=False)
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
df['Mileage'].fillna(df['Mileage'].median(), inplace=True)

In [ ]:
text_cols = ['Manufacturer', 'Model', 'Category', 'Leather interior',
             'Fuel type', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color']

for col in text_cols:
    if col in df.columns and df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.strip().str.lower()

In [ ]:
df['Leather interior'] = df['Leather interior'].map({'yes': 1, 'no': 0})
df['Wheel'] = df['Wheel'].map({'left wheel': 1, 'right-hand drive': 0})

In [ ]:
threshold_manufacturer = 20
manufacturer_counts = df['Manufacturer'].value_counts()
rare_manufacturers = manufacturer_counts[manufacturer_counts < threshold_manufacturer].index
df['Manufacturer'] = df['Manufacturer'].replace(rare_manufacturers, 'other_manufacturer')

df = pd.get_dummies(df, columns=['Manufacturer'], drop_first=True)

In [ ]:
threshold_model = 10
model_counts = df['Model'].value_counts()
rare_models = model_counts[model_counts < threshold_model].index
df['Model'] = df['Model'].replace(rare_models, 'other_model')

df = pd.get_dummies(df, columns=['Model'], drop_first=True)

In [ ]:
one_hot_cols = ['Category', 'Fuel type', 'Gear box type', 'Drive wheels', 'Color']

cols_to_one_hot_now = [col for col in one_hot_cols if col in df.columns and df[col].dtype == 'object']

if cols_to_one_hot_now:
    df = pd.get_dummies(df, columns=cols_to_one_hot_now, drop_first=True)

In [ ]:
print("\n--- Final Check: DataFrame Info after all processing ---")
df.info()
print("\n--- Final Check: First 5 rows of Processed DataFrame ---")
print(df.head())


--- Final Check: DataFrame Info after all processing ---
<class 'pandas.core.frame.DataFrame'>
Index: 18924 entries, 0 to 19236
Columns: 291 entries, Price to Color_yellow
dtypes: bool(282), float64(3), int64(6)
memory usage: 6.5 MB

--- Final Check: First 5 rows of Processed DataFrame ---
   Price    Levy  Prod. year  Leather interior  Engine volume  Mileage  \
0  13328  1399.0        2010                 1            3.5   186005   
1  16621  1018.0        2011                 0            3.0   192000   
2   8467   781.0        2006                 0            1.3   200000   
3   3607   862.0        2011                 1            2.5   168966   
4  11726   446.0        2014                 1            1.3    91901   

   Cylinders  Wheel  Airbags  Manufacturer_bmw  ...  Color_green  Color_grey  \
0        6.0      1       12             False  ...        False       False   
1        6.0      1        8             False  ...        False       False   
2        4.0      0    

In [ ]:
price_threshold = 5000000
df_filtered = df[df['Price'] <= price_threshold].copy()

In [ ]:
X = df_filtered.drop('Price', axis=1)
y = df_filtered['Price']
print("X shape: "+str(X.shape))
print("Y shape: "+str(y.shape))

X shape: (18923, 290)
Y shape: (18923,)


## Data spliting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Data scaling

In [ ]:
#scaler=StandardScaler()

#X_train_scaled=scaler.fit_transform(X_train)
#X_test_scaled=scaler.transform(X_test)

# Data modeling

## Random Forest Regressor

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2 Score): {r2:.4f}")


Mean Absolute Error (MAE): 4502.89
Root Mean Squared Error (RMSE): 10329.73
R-squared (R2 Score): 0.6876


## Xgboost


In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=900, learning_rate=0.1,
                              random_state=42, n_jobs=-1)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)

In [ ]:
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb= mean_squared_error(y_test, y_pred_xgb)
rmse_xgb= np.sqrt(mse)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"Mean Absolute Error (MAE): {mae_xgb:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_xgb:.2f}")
print(f"R2 Score for XGBoost: {r2_xgb:.4f}")


Mean Absolute Error (MAE): 4913.09
Root Mean Squared Error (RMSE): 10329.73
R2 Score for XGBoost: 0.6978
